In [1]:
import torch
import time
import math
import dgl
import numpy as np
import torch as th
from dgl.data import citation_graph as citegrh
from dgl.data import CoraBinary
from dgl.data import CoraGraphDataset
from dgl import DGLGraph
import dgl.function as fn
import networkx as nx
import torch.nn.functional as F
from dgl.data import RedditDataset, KarateClubDataset
from dgl.nn.pytorch.conv import GraphConv
import matplotlib.pyplot as plt
from torch.nn import MSELoss
from losses import compute_loss_multiclass
from functools import partial
import tensorwatch as tw

Using backend: pytorch


In [2]:
import tensorwatch as tw
import time

# streams will be stored in test.log file
w = tw.Watcher(filename='test.log')

# create a stream for logging
s = w.create_stream(name='metric1')

# generate Jupyter Notebook to view real-time streams
w.make_notebook()
class GCN(th.nn.Module):
    def __init__(self,
                 g,
                 in_feats,
                 n_hidden,
                 n_classes,
                 n_layers,
                 activation,
                 dropout):
        super(GCN, self).__init__()
        self.g = g
        self.layers = th.nn.ModuleList()
        # input layer
        self.layers.append(GraphConv(in_feats, n_hidden, activation=activation))
        # n_layers hidden layer
        for i in range(n_layers - 1):
            self.layers.append(GraphConv(n_hidden, n_hidden, activation=activation))
        # 1 output layer, use softmax as activation on this layer
        self.layers.append(GraphConv(n_hidden, n_classes,activation=partial(F.softmax,dim=1)))
        self.dropout = th.nn.Dropout(p=dropout)

    def forward(self, features):
        h = features
        for i, layers in enumerate(self.layers):
            if i != 0:
                h = self.dropout(h)
            h = layers(self.g, h)
        #output is f(x) through a softmax filter
        return h
def load_kara():
    data =KarateClubDataset()
    n_classes = data.num_classes
    g = data[0]
    n_edges=g.number_of_edges()
    n=len(g.ndata['label'])
    labels=g.ndata['label']
    #construct features, train,val,test masks
    g.ndata['feat']=g.adj().to_dense()
    #test=g.adj()
    in_feats=g.adj().shape[1]
    #this features is not efficient
    features=g.ndata['feat']

    return g,features,n_classes,in_feats,n_edges,labels


[Debug][0]: FileStream started : C:\Users\benno\OneDrive\Documents\GitHub\GNN_dev\test.log : t=2.33
[Debug][0]: Notebook created : C:\Users\benno\OneDrive\Documents\GitHub\GNN_dev\test.ipynb : t=2.33


In [3]:
dropout = 0
gpu = 0
lr = 5e-2
n_epochs = 20000
n_hidden =256  # 隐藏层节点的数量
n_layers = 0  # 输入层 + 输出层的数量
weight_decay = 5e-4  # 权重衰减
self_loop = True  # 自循环

g,features,n_classes,in_feats,n_edges,labels = load_kara()

model = GCN(g,
        in_feats,
        n_hidden,
        n_classes,
        n_layers,
        F.relu,
        dropout)


tw.draw_model(model)


AttributeError: module 'torch.onnx' has no attribute 'set_training'